# 🚀 Data Platform クイックスタート

各コンポーネントへの接続方法を説明します。

In [ ]:
import os
import pandas as pd

STORAGE_NODE = os.environ.get('STORAGE_NODE_IP', '10.10.10.10')
DWH_NODE = os.environ.get('DWH_NODE_IP', '10.10.10.20')

print(f"Storage Node: {STORAGE_NODE}")
print(f"DWH Node: {DWH_NODE}")

## MinIO接続（データレイク）

In [ ]:
import boto3
from botocore.client import Config

s3 = boto3.client(
    's3',
    endpoint_url=f'http://{STORAGE_NODE}:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123',
    config=Config(signature_version='s3v4')
)

print("MinIOバケット一覧:")
for bucket in s3.list_buckets()['Buckets']:
    print(f"  - {bucket['Name']}")

## PostgreSQL DWH接続

In [ ]:
from sqlalchemy import create_engine, text

# DWH接続
dwh_engine = create_engine(f'postgresql://analyst:analyst123@{DWH_NODE}:5432/dwh')

with dwh_engine.connect() as conn:
    # PostgreSQLバージョン確認
    result = conn.execute(text('SELECT version()'))
    print(f"PostgreSQL DWH: {result.fetchone()[0][:60]}...")
    
    # スキーマ一覧
    result = conn.execute(text("""
        SELECT schema_name 
        FROM information_schema.schemata 
        WHERE schema_name NOT LIKE 'pg_%' 
        AND schema_name != 'information_schema'
    """))
    print("\nDWHスキーマ:")
    for row in result:
        print(f"  - {row[0]}")

## Gold層からデータ取得例

In [ ]:
# 日付ディメンションテーブルから取得
query = """
SELECT 
    year, 
    month, 
    COUNT(*) as days_count,
    SUM(CASE WHEN is_weekend THEN 1 ELSE 0 END) as weekend_days
FROM gold.dim_date
WHERE year = 2024
GROUP BY year, month
ORDER BY month
"""

df = pd.read_sql(query, dwh_engine)
print("2024年の月別日数:")
df

## Storage Node PostgreSQL接続

In [ ]:
# Storage Node接続
storage_engine = create_engine(f'postgresql://analyst:analyst123@{STORAGE_NODE}:5432/analytics')

with storage_engine.connect() as conn:
    result = conn.execute(text('SELECT version()'))
    print(f"Storage Node PostgreSQL: {result.fetchone()[0][:60]}...")

## 可視化サンプル

In [ ]:
import plotly.express as px
import numpy as np

# サンプルデータ
df_viz = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=30),
    'sales': np.random.randint(100, 500, 30),
    'region': np.random.choice(['東京', '大阪', '福岡'], 30)
})

fig = px.line(df_viz, x='date', y='sales', color='region', title='地域別売上推移')
fig.show()

## 🔗 参考リンク

- **MinIO Console**: http://STORAGE_NODE_IP:9001
- **PostgreSQL DWH**: DWH_NODE_IP:5432
- **Trino**: http://DWH_NODE_IP:8080
- **Airflow**: http://ETL_NODE_IP:8080
- **Superset**: http://ANALYTICS_NODE_IP:8088
- **Metabase**: http://ANALYTICS_NODE_IP:3000